<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

**텐서**
* 동적 크기를 갖는 다차원 자료구조
* 차원 표기법
    * 구조(shape): [D0, D1, ... Dn-1]
    * 랭크(rank): n
    * 차원번호(dimension number): n-D
* 변수와 상수 선언
    * 변수: `tf.variable()`
    * 상수: `tf.constant()`
    * 변수 초기화: `tf.global_variables_initializer()`
    * 심벌릭 변수
        * 프로그램 실행 중 데이터를 바꾸기 위함
        * `ph_1 = tf.placeholder('자료형, 구조`): 심벌릭 변수 선언
        * `sess.run(..., feed_dict={ph_1: a, ph2: b})` 플레이스홀더에 값 입력

In [1]:
import tensorflow as tf

In [11]:
ph_1 = tf.placeholder('float')
ph_2 = tf.placeholder('float')

y = tf.multiply(ph_1, ph_2)  # 연산 등록, 곱하기


sess = tf.Session()
init = tf.global_variables_initializer()

sess.run(init)  # 초기화
sess.run(y, feed_dict={ph_1: 3, ph_2: 4})  # 플레이스홀더에 값 입력

12.0